In [2]:
# Dependencies and Setup
!pip install gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# API keys
# OpenWeatherMap API Key
weather_api_key = "42fc9adc90ea2e2c2fdd3e059f13a8de"
# Google API Key
g_key = "AIzaSyAqKJpKDT8joFxSRi-hVSjmLrx1eHgTZc"

In [3]:
#Bring in cities we used in first part
weather_data = pd.read_csv(r"cities_hw.csv")
weather_data

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Vaini,75,TO,1609800109,88,-21.20,-175.20,82.40,12.75
1,1,Korčula,90,HR,1609800109,100,42.96,17.14,33.80,2.24
2,2,Otavi,96,NaN,1609800109,67,-19.65,17.33,72.70,3.38
3,3,Nome,1,US,1609800110,54,64.50,-165.41,-0.40,10.29
4,4,Healesville,77,AU,1609800110,97,-37.65,145.52,64.00,1.01
...,...,...,...,...,...,...,...,...,...,...
576,576,Grants,1,US,1609800212,23,35.15,-107.85,51.80,8.05
577,577,Hwange,16,ZW,1609800212,89,-18.37,26.50,68.68,0.16
578,578,Hervey Bay,40,AU,1609800212,69,-25.30,152.85,88.00,6.93
579,579,Kumbo,1,CM,1609800213,37,6.20,10.67,62.51,2.77


In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 70
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 60) & (weather_data["Max Temp"] <= 70)].dropna()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,14,Busselton,0,AU,1609800112,76,-33.65,115.33,63.00,7.00
52,52,Aswān,0,EG,1609800118,30,24.09,32.91,62.31,8.61
240,240,Eilat,0,IL,1609800152,67,29.56,34.95,62.60,4.45
333,333,Mandurah,0,AU,1609799928,65,-32.55,115.70,64.99,1.99
373,373,Tessalit,0,ML,1609800176,24,20.20,1.01,64.76,8.25
428,428,Ngunguru,0,NZ,1609800186,68,-35.62,174.50,68.77,7.11
471,471,Nouadhibou,0,MR,1609800194,52,20.93,-17.03,68.00,2.24
530,530,Camargo,0,MX,1609800204,21,27.67,-105.17,68.63,7.49


In [25]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df =  pd.DataFrame(hotel_df)
hotel_df.head()

,City,Country,Lat,Lng
14,Busselton,AU,-33.65,115.33
52,Aswān,EG,24.09,32.91
240,Eilat,IL,29.56,34.95
333,Mandurah,AU,-32.55,115.70
373,Tessalit,ML,20.20,1.01


In [27]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = hotel_df

# Display the result
hotel_df

ValueError: Wrong number of items passed 4, placement implies 1

In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [14]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 14: Busselton.
Missing field/result... skipping.
------------
Retrieving Results for Index 52: Aswān.
Missing field/result... skipping.
------------
Retrieving Results for Index 240: Eilat.
Missing field/result... skipping.
------------
Retrieving Results for Index 333: Mandurah.
Missing field/result... skipping.
------------
Retrieving Results for Index 373: Tessalit.
Missing field/result... skipping.
------------
Retrieving Results for Index 428: Ngunguru.
Missing field/result... skipping.
------------
Retrieving Results for Index 471: Nouadhibou.
Missing field/result... skipping.
------------
Retrieving Results for Index 530: Camargo.
Missing field/result... skipping.
------------
-------End of Search-------


In [15]:
# Display the hotel dataframe
hotel_df
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig